# Immigration Data for the United States
### Data Engineering Capstone Project

#### Project Summary

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [2]:
# Do all imports and installs here
import pandas as pd
from datetime import datetime
import datetime as dt
import numpy as np

## Spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, date_add, col, split, year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql.types import StructType as R, StructField as Fld
from pyspark.sql.types import StringType as Str, DoubleType as Dbl, IntegerType as Int, DateType as Date, LongType as Long
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

### Step 1: Scope the Project and Gather Data

In this capstone project we are focusing on the immigration data for the US. The main idea is to build a broader view of the immigration behaviour to the US by joining it with a series of potentially interesting attributes such as:
- the travel mode, seasonality, reason of travel, origin/destinations and avg. time spent in the US
- the type of airports used to enter the US
- the city demographics to which they're traveling
- temperature statistics about the location where they are traveling from/to

Given the above objective, the end solution will be a set of fact and dimensional tables hosted in Amazon S3 that the final user can easily access and analyse to generate new insights.

An ETL data pipeline will be built to perform the following steps:  
1. Extract data from different sources
2. Process them with the help of Python and Spark
3. Load the final tables into S3 in Parquet format (in this noteboook we will actually first load them locally).


The datasets we will be using are:
- **I94 immigration data**: [this dataset](https://travel.trade.gov/research/reports/i94/historical/2016.html) comes from the US National Tourism and Trade Office and is available locally.
- **Airport Code Table**: this is a simple table of airport codes and corresponding cities. It comes from [here](https://datahub.io/core/airport-codes#data).
- **U.S. City Demographic data**: this dataset comes from [OpenSoft](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/).
- **World Temperature Data**: this dataset came from [Kaggle](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data).

Below we will start by loading and exploring each dataset to have an idea of what steps are needed to run the complete data pipeline.

##### Immigration dataset

Since the complete immigration dataset ss very big, lets first **explore the sample dataset** provided in .csv format

In [2]:
# Read in the immigration sample dataset
df_sample = pd.read_csv('immigration_data_sample.csv')
print(df_sample.shape)

(1000, 29)


In [3]:
pd.set_option('display.max_columns', 30) # extend default pandas max columns
df_sample.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,NaN,NaN,G,O,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,2.0,1.0,20160423,MTR,NaN,G,R,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,76.0,2.0,1.0,20160407,NaN,NaN,G,O,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,25.0,2.0,1.0,20160428,DOH,NaN,G,O,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,19.0,2.0,1.0,20160406,NaN,NaN,Z,K,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


A data dictionary with definitions for each field is available in the provided file _I94_SAS_Labels_descriptions.SAS_ .  


For example, some of the main fields available are:  

* i94yr: 4 digit year
* i94mon: numeric month
* i94cit/i94res: country code of the immigrant (respectively citizenship & residence)
* i94port: arrival airport code
* arrdate: arrival date in the USA in SAS numeric format
* depdate: departure date from the USA in SA numeric format
* i94mode: how the immigrant arrived (air/sea/land)
* i94addr: US state of the immigrant? or of the arrival port? !!!!
* i94visa: visa code collapsed into three categories (1=Business; 2 = Pleasure; 3=Student)
* count: used for summary statistics
* etc.

Lets now try to **read the full dataset** with the help of **Spark**

In [3]:
	
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
    config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11").\
    enableHiveSupport().\
    getOrCreate()

In [5]:
df_spark = spark.read.format('com.github.saurfang.sas.spark').\
    load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

df_spark.count()

3096313

In [6]:
print(df_spark.show(3))
print(df_spark.printSchema())

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-------------+-----+--------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|       admnum|fltno|visatype|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-------------+-----+--------+
|  6.0|2016.0|   4.0| 692.0| 692.0|    XXX|20573.0|   null|   null|   null|  37.0|    2.0|  1.0|    null|    null| null|      T|   null|      U|   null| 1979.0|10282016|  null|  null|   null|1.897628485E9| null|      B2|
|  7.0|2016.0|   4.0| 254.0| 276.0|    ATL|20551.0|    1.0|     AL|   null|  25.0|    3.0|  1.0|20130811|     SEO| n

In [8]:
#write to parquet
##df_spark.write.parquet("sas_data")
##df_spark=spark.read.parquet("sas_data")
df_spark.write.parquet("sas_data_mine")

In [11]:
df_spark2=spark.read.parquet("sas_data_mine")
df_spark2.count()

3096313

In [12]:
df_spark2.show(3)
df_spark2.printSchema()

+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|    cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|5748517.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     CA|20582.0|  40.0|    1.0|  1.0|20160430|     SYD| null|      G|      O|   null|      M| 1976.0|10292016|     F|  null|     QF|9.495387003E10|00011|      B1|
|5748518.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     NV|20591.0|  32.0|    1.0|  

##### US Cities Demographics dataset

In [7]:
# Read in the data here
df_demographics = pd.read_csv('us-cities-demographics.csv', sep=';')
print(df_demographics.shape)

(2891, 12)


In [8]:
df_demographics.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


This dataset contains information about the **demographics of all US cities** and census-designated places with a population greater or equal to 65,000. More info [here](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/information/?dataChart=eyJxdWVyaWVzIjpbeyJjb25maWciOnsiZGF0YXNldCI6InVzLWNpdGllcy1kZW1vZ3JhcGhpY3MiLCJvcHRpb25zIjp7fX0sImNoYXJ0cyI6W3siYWxpZ25Nb250aCI6dHJ1ZSwidHlwZSI6ImNvbHVtbiIsImZ1bmMiOiJBVkciLCJ5QXhpcyI6Im1lZGlhbl9hZ2UiLCJzY2llbnRpZmljRGlzcGxheSI6dHJ1ZSwiY29sb3IiOiIjRkY1MTVBIn1dLCJ4QXhpcyI6ImNpdHkiLCJtYXhwb2ludHMiOjUwLCJzb3J0IjoiIn1dLCJ0aW1lc2NhbGUiOiIiLCJkaXNwbGF5TGVnZW5kIjp0cnVlLCJhbGlnbk1vbnRoIjp0cnVlfQ%3D%3D) .

_Could be interesting to join this info with the immigration table above to better understand immigration flows and its impact on a city/state level. We should be able to join the two datasets via the US state code fields._

##### Airport dataset

In [9]:
# Read in the data here
df_airports = pd.read_csv('airport-codes_csv.csv')
print(df_airports.shape)

(55075, 12)


In [10]:
df_airports.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


This is a simple table of airport codes and corresponding cities. More info [here](https://datahub.io/core/airport-codes#data)

We can get more info about each airport (type, geographical coordinates, etc.). Again we should be able to join it with the immigration dataset via the US state code.

##### World temperature dataset

In [11]:
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
df_temperature = pd.read_csv(fname)
print(df_temperature.shape)

(8599212, 7)


In [12]:
df_temperature.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


This dataset provides contains land temperatures by city. More info [here](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data)

_Might be interesting to analyse immigration flows by the temparature dimensions: e.g. are travelers with Tourism visa more likely to travel to a warmer place? What about business visa immigrants? We should be able to join this dataset with the immigration one via city/country fields._

##### Mapping codes

Based on the immigration data dictionary file, I've also created and uploaded a couple of files providing the mappings for:  

* country
* arrival port
* visa
* mode of travel
* state

Lets read them in

In [13]:
map_country_codes = pd.read_csv('map_codes/map_country_codes.csv')
print(map_country_codes.head())
map_port_codes = pd.read_csv('map_codes/map_port_codes.csv')
print(map_port_codes.head())

map_mode_codes = pd.read_csv('map_codes/map_mode_codes.csv')
print(map_mode_codes.head())
map_visa_codes = pd.read_csv('map_codes/map_visa_codes.csv')
print(map_visa_codes.head())
map_state_codes = pd.read_csv('map_codes/map_state_codes.csv')
print(map_state_codes.head())

   code      country
0   582      MEXICO 
1   236  AFGHANISTAN
2   101      ALBANIA
3   316      ALGERIA
4   102      ANDORRA
  code                  location state
0  ALC                     ALCAN    AK
1  ANC                 ANCHORAGE    AK
2  BAR  BAKER AAF - BAKER ISLAND    AK
3  DAC             DALTONS CACHE    AK
4  PIZ    DEW STATION PT LAY DEW    AK
   code          mode
0     1           Air
1     2           Sea
2     3          Land
3     4  Not Reported
   code      visa
0     1  Business
1     2  Pleasure
2     3   Student
  code       state
0   AL     ALABAMA
1   AK      ALASKA
2   AZ     ARIZONA
3   AR    ARKANSAS
4   CA  CALIFORNIA


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

##### US Cities Demographics

In [8]:
df_demographics = pd.read_csv('us-cities-demographics.csv', sep=';')
print(df_demographics.shape)
df_demographics.head()

(2891, 12)


,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [9]:
# check missing values
df_demographics.isnull().sum()

City                       0
State                      0
Median Age                 0
Male Population            3
Female Population          3
Total Population           0
Number of Veterans        13
Foreign-born              13
Average Household Size    16
State Code                 0
Race                       0
Count                      0
dtype: int64

In [10]:
# Clearly is not just by City
df_demographics['City'].nunique()

567

In [11]:
#Let see see if we remove duplicated rows by adding State and Race
print(df_demographics[df_demographics[['City', 'State']].duplicated()].shape)
print(df_demographics[df_demographics[['City', 'State', 'Race']].duplicated()].shape) # yes

(2295, 12)
(0, 12)


The table is aggegated by a combination of these 3 variables:  
* City
* State Code
* Race

In [12]:
# Read it using Spark
df_demographics = spark.read.csv('us-cities-demographics.csv', header = 'True', sep=";")

In [13]:
df_demographics.show(3)
df_demographics.printSchema()

+-------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+------------------+-----+
|         City|        State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|State Code|              Race|Count|
+-------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+------------------+-----+
|Silver Spring|     Maryland|      33.8|          40601|            41862|           82463|              1562|       30908|                   2.6|        MD|Hispanic or Latino|25924|
|       Quincy|Massachusetts|      41.0|          44129|            49500|           93629|              4147|       32935|                  2.39|        MA|             White|58723|
|       Hoover|      Alabama|      38.5|          38040|            46799|           

In [14]:
df_demographics_clean = df_demographics.select(
    col('City').alias('city'),
    col('State').alias('state_name'),
    col('Median Age').alias('median_age'),
    col('Male Population').alias('male_population'),
    col('Female Population').alias('female_population'),
    col('Number of Veterans').alias('number_veterans'),
    col('Foreign-born').alias('foreign_born'),
    col('Average Household Size').alias('avg_household_size'),
    col('State Code').alias('state_code'),
    col('Race').alias('race'),
    col('Count').alias('count'),
)

df_demographics_clean.printSchema()

root
 |-- city: string (nullable = true)
 |-- state_name: string (nullable = true)
 |-- median_age: string (nullable = true)
 |-- male_population: string (nullable = true)
 |-- female_population: string (nullable = true)
 |-- number_veterans: string (nullable = true)
 |-- foreign_born: string (nullable = true)
 |-- avg_household_size: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- race: string (nullable = true)
 |-- count: string (nullable = true)



##### US Airports

In [19]:
print(df_airports.shape)
df_airports.head()

(55075, 12)


,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [20]:
# check missing values
df_airports.isnull().sum()

ident               0
type                0
name                0
elevation_ft     7006
continent       27719
iso_country       247
iso_region          0
municipality     5676
gps_code        14045
iata_code       45886
local_code      26389
coordinates         0
dtype: int64

In [21]:
df_airports['iso_country'].value_counts().head()

US    22757
BR     4334
CA     2784
AU     1963
KR     1376
Name: iso_country, dtype: int64

In [22]:
df_airports_us = df_airports.query("iso_country == 'US' ")

In [23]:
print(df_airports_us.shape)
df_airports_us.isnull().sum()

(22757, 12)


ident               0
type                0
name                0
elevation_ft      239
continent       22756
iso_country         0
iso_region          0
municipality      102
gps_code         1773
iata_code       20738
local_code       1521
coordinates         0
dtype: int64

In [24]:
df_airports_us = df_airports_us[df_airports_us['municipality'].notna()]
print(df_airports_us.shape)

(22655, 12)


In [25]:
df_airports = spark.read.csv('airport-codes_csv.csv', header = 'True')

In [26]:
df_airports.show(3)
df_airports.printSchema()

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|  00A|     heliport|   Total Rf Heliport|          11|       NA|         US|     US-PA|    Bensalem|     00A|     null|       00A|-74.9336013793945...|
| 00AA|small_airport|Aero B Ranch Airport|        3435|       NA|         US|     US-KS|       Leoti|    00AA|     null|      00AA|-101.473911, 38.7...|
| 00AK|small_airport|        Lowell Field|         450|       NA|         US|     US-AK|Anchor Point|    00AK|     null|      00AK|-151.695999146, 5...|
+-----+-------------+--------------------+------------+---------+-----------+-----

In [27]:
df_airports_us_clean = df_airports.filter("iso_country == 'US'")\
    .withColumn("state", split(col("iso_region"), "-")[1])\
    .withColumn("latitude", split(col("coordinates"), ",")[0].cast(Dbl()))\
    .withColumn("longitude", split(col("coordinates"), ",")[1].cast(Dbl()))

print(df_airports_us_clean.count())
df_airports_us_clean.show(3)
df_airports_us_clean.printSchema()

22757
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+-----+------------------+--------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|state|          latitude|     longitude|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+-----+------------------+--------------+
|  00A|     heliport|   Total Rf Heliport|          11|       NA|         US|     US-PA|    Bensalem|     00A|     null|       00A|-74.9336013793945...|   PA|-74.93360137939453|40.07080078125|
| 00AA|small_airport|Aero B Ranch Airport|        3435|       NA|         US|     US-KS|       Leoti|    00AA|     null|      00AA|-101.473911, 38.7...|   KS|       -101.473911|     38.704022|
| 00AK|small_airport|        

##### Immigration data

In [28]:
#pd.set_option('display.max_columns', 30) # extend default pandas max columns
df_sample.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,NaN,NaN,G,O,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,2.0,1.0,20160423,MTR,NaN,G,R,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,76.0,2.0,1.0,20160407,NaN,NaN,G,O,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,25.0,2.0,1.0,20160428,DOH,NaN,G,O,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,19.0,2.0,1.0,20160406,NaN,NaN,Z,K,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


In [29]:
df_sample.isnull().sum()

Unnamed: 0       0
cicid            0
i94yr            0
i94mon           0
i94cit           0
i94res           0
i94port          0
arrdate          0
i94mode          0
i94addr         59
depdate         49
i94bir           0
i94visa          0
count            0
dtadfile         0
visapost       618
occup          996
entdepa          0
entdepd         46
entdepu       1000
matflag         46
biryear          0
dtaddto          0
gender         141
insnum         965
airline         33
admnum           0
fltno            8
visatype         0
dtype: int64

In [30]:
print(df_spark.show(3, truncate = False))
print(df_spark.printSchema())

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-------------+-----+--------+
|cicid|i94yr |i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear|dtaddto |gender|insnum|airline|admnum       |fltno|visatype|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-------------+-----+--------+
|6.0  |2016.0|4.0   |692.0 |692.0 |XXX    |20573.0|null   |null   |null   |37.0  |2.0    |1.0  |null    |null    |null |T      |null   |U      |null   |1979.0 |10282016|null  |null  |null   |1.897628485E9|null |B2      |
|7.0  |2016.0|4.0   |254.0 |276.0 |ATL    |20551.0|1.0    |AL     |null   |25.0  |3.0    |1.0  |20130811|SEO     |nu

In [4]:
df_spark_sample = spark.read.csv('immigration_data_sample.csv', header = 'True')
df_spark_sample.show(1)
print(df_spark_sample.printSchema())

+-------+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-------------+-----+--------+
|    _c0|    cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|       admnum|fltno|visatype|
+-------+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-------------+-----+--------+
|2027561|4084316.0|2016.0|   4.0| 209.0| 209.0|    HHW|20566.0|    1.0|     HI|20573.0|  61.0|    2.0|  1.0|20160422|    null| null|      G|      O|   null|      M| 1955.0|07202016|     F|  null|     JL|56582674633.0|00782|      WT|
+-------+---------+------+------+------+------+-------+-------+-----

In [32]:
#read in mapping files with Spark
map_mode_codes = spark.read.csv('map_codes/map_mode_codes.csv', header = 'True')
map_mode_codes.show(3)
map_visa_codes = spark.read.csv('map_codes/map_visa_codes.csv', header = 'True')
map_visa_codes.show(3)
map_state_codes = spark.read.csv('map_codes/map_state_codes.csv', header = 'True')
map_state_codes.show(3)

map_port_codes = spark.read.csv('map_codes/map_port_codes.csv', header = 'True')
map_port_codes.show(3)
map_country_codes = spark.read.csv('map_codes/map_country_codes.csv', header = 'True')
map_country_codes.show(3)


+----+----+
|code|mode|
+----+----+
|   1| Air|
|   2| Sea|
|   3|Land|
+----+----+
only showing top 3 rows

+----+--------+
|code|    visa|
+----+--------+
|   1|Business|
|   2|Pleasure|
|   3| Student|
+----+--------+

+----+-------+
|code|  state|
+----+-------+
|  AL|ALABAMA|
|  AK| ALASKA|
|  AZ|ARIZONA|
+----+-------+
only showing top 3 rows

+----+--------------------+-----+
|code|            location|state|
+----+--------------------+-----+
| ALC|               ALCAN|   AK|
| ANC|           ANCHORAGE|   AK|
| BAR|BAKER AAF - BAKER...|   AK|
+----+--------------------+-----+
only showing top 3 rows

+----+-----------+
|code|    country|
+----+-----------+
| 582|    MEXICO |
| 236|AFGHANISTAN|
| 101|    ALBANIA|
+----+-----------+
only showing top 3 rows



In [34]:
#df_spark_sample.createOrReplaceTempView("df_immigration")  # to use sample dataset
df_spark.createOrReplaceTempView("df_immigration")  # to use full dataset
map_mode_codes.createOrReplaceTempView("map_mode_codes")
map_visa_codes.createOrReplaceTempView("map_visa_codes")
map_state_codes.createOrReplaceTempView("map_state_codes")

map_port_codes.createOrReplaceTempView("map_port_codes")
map_country_codes.createOrReplaceTempView("map_country_codes")

In [35]:
#Try to query via sql
spark.sql("""
SELECT count(*)
FROM df_immigration
""").show()

spark.sql("""
SELECT *
FROM df_immigration
limit 3
""").show()

spark.sql("""
SELECT *
FROM map_mode_codes
limit 3
""").show()

spark.sql("""
SELECT *
FROM map_visa_codes
limit 3
""").show()

spark.sql("""
SELECT *
FROM map_state_codes
limit 3
""").show()

spark.sql("""
SELECT *
FROM map_port_codes
limit 3
""").show()

spark.sql("""
SELECT *
FROM map_country_codes
limit 3
""").show()

+--------+
|count(1)|
+--------+
| 3096313|
+--------+

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-------------+-----+--------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|       admnum|fltno|visatype|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-------------+-----+--------+
|  6.0|2016.0|   4.0| 692.0| 692.0|    XXX|20573.0|   null|   null|   null|  37.0|    2.0|  1.0|    null|    null| null|      T|   null|      U|   null| 1979.0|10282016|  null|  null|   null|1.897628485E9| null|      B2|
|  7.0|2016.0|   4.0| 254.0| 276.0|    ATL|20551.0|    1.0| 

In [36]:
df_immigration_joined = spark.sql("""
    SELECT
        i.i94yr as year,
        i.i94mon as month,
        i.i94cit as citizenship_country,
        i.i94res as residence_country,
        i.i94port as port,
        i.arrdate,
        date_add(to_date('1960-01-01'), i.arrdate) AS arrival_date,
        coalesce(m.mode, 'Not reported') as arrival_mode,
        coalesce(s.code, 'Not reported') as us_state,
        i.depdate,
        date_add(to_date('1960-01-01'), i.depdate) AS departure_date,
        i.i94bir as respondent_age,
        coalesce(v.visa, 'Not reported') as visa,
        i.dtadfile as date_added,
        i.visapost as visa_issued_department,
        i.occup as occupation,
        i.entdepa as arrival_flag,
        i.entdepd as departure_flag,
        i.entdepu as update_flag,
        i.matflag as match_arrival_departure_fag,
        i.biryear as birth_year,
        i.dtaddto as allowed_to_stay_date,
        i.insnum as ins_number,
        i.airline as airline,
        i.admnum as admission_number,
        i.fltno as flight_number,
        i.visatype as visa_type
    from df_immigration i 
    left join map_mode_codes m on i.i94mode = m.code
    left join map_visa_codes v on i.i94visa = v.code
    left join map_state_codes s on i.i94addr = s.code
""")

In [37]:
df_immigration_joined.show(3)
df_immigration_joined.printSchema()

+------+-----+-------------------+-----------------+----+-------+------------+------------+------------+-------+--------------+--------------+--------+----------+----------------------+----------+------------+--------------+-----------+---------------------------+----------+--------------------+----------+-------+----------------+-------------+---------+
|  year|month|citizenship_country|residence_country|port|arrdate|arrival_date|arrival_mode|    us_state|depdate|departure_date|respondent_age|    visa|date_added|visa_issued_department|occupation|arrival_flag|departure_flag|update_flag|match_arrival_departure_fag|birth_year|allowed_to_stay_date|ins_number|airline|admission_number|flight_number|visa_type|
+------+-----+-------------------+-----------------+----+-------+------------+------------+------------+-------+--------------+--------------+--------+----------+----------------------+----------+------------+--------------+-----------+---------------------------+----------+-----------

In [38]:
df_immigration_joined.createOrReplaceTempView("df_immigration_joined")

In [39]:
# Look into some numerical variables, and in particular the calculated dates
spark.sql("""
SELECT year, month, arrdate, arrival_date, depdate, departure_date, respondent_age, birth_year
FROM df_immigration_joined
limit 5
""").show()

+------+-----+-------+------------+-------+--------------+--------------+----------+
|  year|month|arrdate|arrival_date|depdate|departure_date|respondent_age|birth_year|
+------+-----+-------+------------+-------+--------------+--------------+----------+
|2016.0|  4.0|20573.0|  2016-04-29|   null|          null|          37.0|    1979.0|
|2016.0|  4.0|20551.0|  2016-04-07|   null|          null|          25.0|    1991.0|
|2016.0|  4.0|20545.0|  2016-04-01|20691.0|    2016-08-25|          55.0|    1961.0|
|2016.0|  4.0|20545.0|  2016-04-01|20567.0|    2016-04-23|          28.0|    1988.0|
|2016.0|  4.0|20545.0|  2016-04-01|20567.0|    2016-04-23|           4.0|    2012.0|
+------+-----+-------+------------+-------+--------------+--------------+----------+



In [40]:
spark.sql("""
    SELECT COUNT(*)
    FROM df_immigration_joined
    WHERE arrival_date > departure_date
""").show()

+--------+
|count(1)|
+--------+
|     375|
+--------+



In [42]:
# Check other fields

# how many recors have either arrival or departure date NULL: 4% of total records
spark.sql("""
    SELECT COUNT(*)
    FROM df_immigration_joined
    WHERE arrival_date is null
    or departure_date is null
""").show()

# admission number might be a unique identifier for the table
spark.sql("""
SELECT COUNT (DISTINCT admission_number)
FROM df_immigration_joined
""").show()

# check quality of port field: should be 3 characters long
spark.sql("""
SELECT LENGTH(port) as num_char
FROM df_immigration_joined
GROUP BY num_char
""").show()

# Only data from April 2016 as expected
spark.sql("""
SELECT min(arrival_date), max(arrival_date), min(departure_date), max(departure_date)
FROM df_immigration_joined
""").show()



+--------+
|count(1)|
+--------+
|  142457|
+--------+

+--------------------------------+
|count(DISTINCT admission_number)|
+--------------------------------+
|                         3075579|
+--------------------------------+

+--------+
|num_char|
+--------+
|       3|
+--------+

+-----------------+-----------------+-------------------+-------------------+
|min(arrival_date)|max(arrival_date)|min(departure_date)|max(departure_date)|
+-----------------+-----------------+-------------------+-------------------+
|       2016-04-01|       2016-04-30|         2001-07-20|         2084-05-16|
+-----------------+-----------------+-------------------+-------------------+



In [43]:
# Lets maybe provide codes for these too, as we did for state
spark.sql("""
SELECT citizenship_country, count(*) as n
FROM df_immigration_joined
GROUP BY citizenship_country
order by n desc
""").show()

spark.sql("""
SELECT residence_country, count(*) as n
FROM df_immigration_joined
GROUP BY residence_country
order by n desc
""").show()

+-------------------+------+
|citizenship_country|     n|
+-------------------+------+
|              135.0|360157|
|              209.0|206873|
|              245.0|191425|
|              111.0|188766|
|              582.0|175781|
|              148.0|157806|
|              254.0|137735|
|              689.0|129833|
|              213.0|110691|
|              438.0|109884|
|              117.0| 78535|
|              123.0| 76920|
|              687.0| 69853|
|              129.0| 57224|
|              691.0| 54120|
|              130.0| 45269|
|              251.0| 41744|
|              692.0| 41349|
|              252.0| 41132|
|              696.0| 40785|
+-------------------+------+
only showing top 20 rows

+-----------------+------+
|residence_country|     n|
+-----------------+------+
|            135.0|368421|
|            209.0|249167|
|            245.0|185609|
|            111.0|185339|
|            582.0|179603|
|            112.0|156613|
|            276.0|136312|
|       

In [44]:
# we could filter out kids? But maybe parents filled out the immigration questionnaire on their behalf, so do not do anything on these. Just remove if there happen to be any negative age.
df_immigration_joined.describe(['respondent_age']).show()

+-------+------------------+
|summary|    respondent_age|
+-------+------------------+
|  count|           3095511|
|   mean|41.767614458485205|
| stddev| 17.42026053458826|
|    min|              -3.0|
|    max|             114.0|
+-------+------------------+



In [45]:
df_immigration_cleaned = spark.sql("""
    select *
    from df_immigration_joined
    where 1=1
        and respondent_age >= 0
        and arrival_date IS NOT NULL 
        and departure_date IS NOT NULL
        and arrival_date <= departure_date
""")

df_immigration_cleaned.count()

2953435

In [46]:
fact_immigration = df_immigration_cleaned.\
    drop('arrdate', 'depdate')
fact_immigration.printSchema()

root
 |-- year: double (nullable = true)
 |-- month: double (nullable = true)
 |-- citizenship_country: double (nullable = true)
 |-- residence_country: double (nullable = true)
 |-- port: string (nullable = true)
 |-- arrival_date: date (nullable = true)
 |-- arrival_mode: string (nullable = false)
 |-- us_state: string (nullable = false)
 |-- departure_date: date (nullable = true)
 |-- respondent_age: double (nullable = true)
 |-- visa: string (nullable = false)
 |-- date_added: string (nullable = true)
 |-- visa_issued_department: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- arrival_flag: string (nullable = true)
 |-- departure_flag: string (nullable = true)
 |-- update_flag: string (nullable = true)
 |-- match_arrival_departure_fag: string (nullable = true)
 |-- birth_year: double (nullable = true)
 |-- allowed_to_stay_date: string (nullable = true)
 |-- ins_number: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admission_number: dou

##### Generate a dimension time table using the Immigration fact table

In [47]:
fact_immigration.createOrReplaceTempView("fact_immigration")

In [48]:
dim_time = spark.sql("""
    SELECT DISTINCT arrival_date AS date
    FROM fact_immigration
    WHERE arrival_date IS NOT NULL
    UNION
    SELECT DISTINCT departure_date AS date
    FROM fact_immigration
    WHERE departure_date IS NOT NULL
""")

dim_time.createOrReplaceTempView("dim_time")

In [49]:
dim_time.show(5)

+----------+
|      date|
+----------+
|2016-04-25|
|2016-05-03|
|2016-08-31|
|2016-07-26|
|2016-08-15|
+----------+
only showing top 5 rows



In [50]:
dim_time = dim_time.select(
    col('date').alias('date'),
    dayofmonth('date').alias('day'),
    weekofyear('date').alias('week'),
    month('date').alias('month'),
    year('date').alias('year')
)

dim_time.show(5)
dim_time.count()

+----------+---+----+-----+----+
|      date|day|week|month|year|
+----------+---+----+-----+----+
|2016-04-25| 25|  17|    4|2016|
|2016-05-03|  3|  18|    5|2016|
|2016-08-31| 31|  35|    8|2016|
|2016-07-26| 26|  30|    7|2016|
|2016-08-15| 15|  33|    8|2016|
+----------+---+----+-----+----+
only showing top 5 rows



175

##### Temperature data

In [101]:
print(df_temperature.shape)
df_temperature.head()

(8599212, 7)


,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [102]:
print(df_temperature.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599212 entries, 0 to 8599211
Data columns (total 7 columns):
dt                               object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                             object
Country                          object
Latitude                         object
Longitude                        object
dtypes: float64(2), object(5)
memory usage: 459.2+ MB
None


In [108]:
print(df_temperature['dt'].min())
print(df_temperature['dt'].max())

1743-11-01
2013-09-01


In [103]:
# Check missing values
df_temperature.isnull().sum()

dt                                    0
AverageTemperature               364130
AverageTemperatureUncertainty    364130
City                                  0
Country                               0
Latitude                              0
Longitude                             0
dtype: int64

In [ ]:
# Which countries have more observations?
df_temperature['Country'].value_counts().head(10)

In [44]:
#Let see see if we remove duplicated rows by adding variables
print(df_temperature[df_temperature[['dt', 'City', 'Country']].duplicated()].shape)
print(df_temperature[df_temperature[['dt', 'City', 'Country', 'Latitude', 'Longitude']].duplicated()].shape) #yes

(46034, 7)
(0, 7)


In [4]:
# Read it into Spark
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
df_temperature = spark.read.csv(fname, header = 'True')

df_temperature.createOrReplaceTempView("df_temperature")

In [37]:
print(df_temperature.count())
df_temperature.show(3)
df_temperature.printSchema()

8599212
+----------+------------------+-----------------------------+-----+-------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty| City|Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-----+-------+--------+---------+
|1743-11-01|             6.068|           1.7369999999999999|Århus|Denmark|  57.05N|   10.33E|
|1743-12-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-01-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
+----------+------------------+-----------------------------+-----+-------+--------+---------+
only showing top 3 rows

root
 |-- dt: string (nullable = true)
 |-- AverageTemperature: string (nullable = true)
 |-- AverageTemperatureUncertainty: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = tr

In [9]:
dim_world_temperatures = spark.sql("""
SELECT 
    dt as date,
    City as city,
    Country as country,
    avg(AverageTemperature) as avg_temperature,
    avg(AverageTemperatureUncertainty) as avg_temperature_uncertainty
    --count(*)
FROM df_temperature
WHERE 1=1
    and dt >='1960-01-01'
    --and dt >= (select min(date) from dim_time) --make sure we cover all dates we have in the fact table
    and AverageTemperature is not null
group by date, city, country
--limit 10
""")

In [10]:
print(dim_world_temperatures.count())
dim_world_temperatures.show(5)
dim_world_temperatures.printSchema()

2247991
+----------+-----+-------+-------------------+---------------------------+
|      date| city|country|    avg_temperature|avg_temperature_uncertainty|
+----------+-----+-------+-------------------+---------------------------+
|1976-01-01|Århus|Denmark|0.14699999999999994|                       0.08|
|1969-11-01|Çorlu| Turkey|             12.213|                      0.225|
|1981-05-01|Çorlu| Turkey| 14.369000000000002|                       0.21|
|1992-07-01|Çorlu| Turkey| 21.534000000000002|        0.28800000000000003|
|1969-08-01|Çorum| Turkey| 20.980999999999998|                      0.308|
+----------+-----+-------+-------------------+---------------------------+
only showing top 5 rows

root
 |-- date: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- avg_temperature: double (nullable = true)
 |-- avg_temperature_uncertainty: double (nullable = true)



### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model



![data_model](img/data_model.png)

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

**Fact Immigration**  
* add mapping codes for state, visa, travel mode (port and country codes are kept in separate tables)
* convert SAS date fields from numeric to datetime format
* remove unnecessary fields: count, original SAS date fields and possibly others
* create final fact table
* write immigration fact table to Parquet

**Time Dimension**  
* extract distinct datetimes from fact table and store it in a time dimension table
* add other complementary fields as year, month, week, etc.
* write time dimension table to Parquet

**Dimension Demographics**
* It's only US cities
* no further cleaning needed
* create demographics table
* write demographics table to Parquet

**Dimension Airports**  
* It's world airports: keep only US airports
* remove rowd with missing values in 'municipality' field
* extract state from 'iso_region' field
* extract latitude/longitude from 'coordinates' field
* create airports dimension table
* write airports dimension table to Parquet

**Dimension Temperature**  
* remove rows with missing 'AverageTemperature' (we could have imputed missing values with closest ones in terms of date/city or even lat/long, but in this instance I think could be risky as it would much deeper investigation to understand for example if there are big period gaps in terms of days for a specific City, which could result in a very imprecise assignation)
- remove latitude/longitude fields
- remove dates prior to 1960 (when [mass air travel began](https://www.insider.com/air-travel-in-every-decade-2017-8) ). The alternative is to keep only dates greater than min(date) in dim_time table, to make sure to cover all dates included in fact table (I commented this condition below, but not included for now as our fact table has only 2016 data).
- compute avg. statistics by day/city/country

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

Below we organised the code into several function and tested one by one (output tables are written locally instead of into S3). However you can find the **whole ETL pipeline** in the provided **etl.py** file.

In [2]:
def create_spark_session():
    """Create or retrieve a Spark session
    """
    
    spark = SparkSession.builder.\
        config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11").\
        enableHiveSupport().\
        getOrCreate()
    return spark

In [22]:
def process_immigration_data(spark, input_data, output_data):
    """Extract immigration data from provided folder in local machine, process it using Spark and 
    loads it into S3 as a set of tables in parquet format:
        - a fact table with immigration data and 
        - a dimension table with correspondent dates available
    @spark: spark session defined previously
    @input_data: location path for the dataset
    @output_data: S3 bucket name where to load processed output data
    """
    
    
    # Read in immigration dataset
    print('Loading immigration data...')
    #df_spark = spark.read.csv('immigration_data_sample.csv', header = 'True') # to use sample dataset
    df_spark = spark.read.format('com.github.saurfang.sas.spark').\
        load(input_data)

    
    # Read in mapping codes datasets
    print('Loading mapping codes...')
    map_mode_codes = spark.read.csv('map_codes/map_mode_codes.csv', header = 'True')
    map_visa_codes = spark.read.csv('map_codes/map_visa_codes.csv', header = 'True')
    map_state_codes = spark.read.csv('map_codes/map_state_codes.csv', header = 'True')

    map_port_codes = spark.read.csv('map_codes/map_port_codes.csv', header = 'True')
    map_country_codes = spark.read.csv('map_codes/map_country_codes.csv', header = 'True')
    
    # Create temporary tables to be able to use SparkSQL
    print('Processing immigration data...')
    #df_spark_sample.createOrReplaceTempView("df_immigration")  # to use sample dataset
    df_spark.createOrReplaceTempView("df_immigration")  # to use full dataset
    map_mode_codes.createOrReplaceTempView("map_mode_codes")
    map_visa_codes.createOrReplaceTempView("map_visa_codes")
    map_state_codes.createOrReplaceTempView("map_state_codes")

    #map_port_codes.createOrReplaceTempView("map_port_codes")
    #map_country_codes.createOrReplaceTempView("map_country_codes")
    
    # Join immigration data with mapping codes, convert SAS dates and rename fields
    df_immigration_joined = spark.sql("""
        SELECT
            i.i94yr as year,
            i.i94mon as month,
            i.i94cit as citizenship_country,
            i.i94res as residence_country,
            i.i94port as port,
            i.arrdate,
            date_add(to_date('1960-01-01'), i.arrdate) AS arrival_date,
            coalesce(m.mode, 'Not reported') as arrival_mode,
            coalesce(s.code, 'Not reported') as us_state,
            i.depdate,
            date_add(to_date('1960-01-01'), i.depdate) AS departure_date,
            i.i94bir as respondent_age,
            coalesce(v.visa, 'Not reported') as visa,
            i.dtadfile as date_added,
            i.visapost as visa_issued_department,
            i.occup as occupation,
            i.entdepa as arrival_flag,
            i.entdepd as departure_flag,
            i.entdepu as update_flag,
            i.matflag as match_arrival_departure_fag,
            i.biryear as birth_year,
            i.dtaddto as allowed_to_stay_date,
            i.insnum as ins_number,
            i.airline as airline,
            i.admnum as admission_number,
            i.fltno as flight_number,
            i.visatype as visa_type
        from df_immigration i 
        left join map_mode_codes m on i.i94mode = m.code
        left join map_visa_codes v on i.i94visa = v.code
        left join map_state_codes s on i.i94addr = s.code
    """)
    
    # Additional cleaning and prepare final fact table
    df_immigration_joined.createOrReplaceTempView("df_immigration_joined")
    df_immigration_cleaned = spark.sql("""
        select *
        from df_immigration_joined
        where 1=1
            and respondent_age >= 0
            and arrival_date IS NOT NULL 
            and departure_date IS NOT NULL
            and arrival_date <= departure_date
    """)
    
    fact_immigration = df_immigration_cleaned.\
        drop('arrdate', 'depdate')
    
    # Write fact table into S3 in parquet format
    print('Writing immigration fact table in parquet...')
    
    fact_immigration.write.partitionBy("year", "month", "us_state").parquet((output_data+"fact_immigration"),'overwrite')
    print('Fact immigration table created in parquet')
    
    # Generate a dimension time table using the Immigration fact table
    print('Extracting time data...')
    fact_immigration.createOrReplaceTempView("fact_immigration")
    dim_time = spark.sql("""
        SELECT DISTINCT arrival_date AS date
        FROM fact_immigration
        WHERE arrival_date IS NOT NULL
        UNION
        SELECT DISTINCT departure_date AS date
        FROM fact_immigration
        WHERE departure_date IS NOT NULL
    """)

    dim_time.createOrReplaceTempView("dim_time")
    
    
    dim_time = dim_time.select(
        col('date').alias('date'),
        dayofmonth('date').alias('day'),
        weekofyear('date').alias('week'),
        month('date').alias('month'),
        year('date').alias('year')
    )
    
    # Write dim time table into S3 in parquet format
    print('Writing time dimension table in parquet...')
    dim_time.write.parquet((output_data+"dim_time"),'overwrite')
    print('Time dimension table created in parquet')
    
    # Write also mapping tables for port_codes and country_codes in parquet format: useful to join by city/country
    print('Writing mapping tables in parquet...')
    map_port_codes.write.parquet((output_data+"map_port_codes"),'overwrite')
    map_country_codes.write.parquet((output_data+"map_country_codes"),'overwrite')
    map_state_codes.write.parquet((output_data+"map_state_codes"),'overwrite')
    print('Mapping tables created in parquet')

In [23]:
spark = create_spark_session()
input_data = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
output_data = "2016_04/"
process_immigration_data(spark=spark, input_data=input_data, output_data=output_data)

Loading immigration data...
Loading mapping codes...
Processing immigration data...
Writing immigration fact table in parquet...
Fact immigration table created in parquet
Extracting time data...
Writing time dimension table in parquet...
Time dimension table created in parquet


In [24]:
#read fact table 
table=spark.read.parquet("2016_04/fact_immigration")
print(table.count())
table.show(3)

2953435
+-------------------+-----------------+----+------------+------------+--------------+--------------+--------+----------+----------------------+----------+------------+--------------+-----------+---------------------------+----------+--------------------+----------+-------+----------------+-------------+---------+------+-----+--------+
|citizenship_country|residence_country|port|arrival_date|arrival_mode|departure_date|respondent_age|    visa|date_added|visa_issued_department|occupation|arrival_flag|departure_flag|update_flag|match_arrival_departure_fag|birth_year|allowed_to_stay_date|ins_number|airline|admission_number|flight_number|visa_type|  year|month|us_state|
+-------------------+-----------------+----+------------+------------+--------------+--------------+--------+----------+----------------------+----------+------------+--------------+-----------+---------------------------+----------+--------------------+----------+-------+----------------+-------------+---------+----

In [25]:
#read dim time table 
table=spark.read.parquet("2016_04/dim_time")
print(table.count())
table.show(3)

175
+----------+---+----+-----+----+
|      date|day|week|month|year|
+----------+---+----+-----+----+
|2016-06-30| 30|  26|    6|2016|
|2016-06-03|  3|  22|    6|2016|
|2016-07-20| 20|  29|    7|2016|
+----------+---+----+-----+----+
only showing top 3 rows



In [10]:
def process_airports_data(spark, input_data, output_data):
    """Extract airport data from provided folder in local machine, process it using Spark and 
    loads it into S3 as a dimension table in parquet format:
    @spark: spark session defined previously
    @input_data: location path for the dataset
    @output_data: S3 bucket name where to load processed output data
    """
        
    # Read in airport data
    print('Loading airport data...')
    df_airports = spark.read.csv(input_data, header = 'True')
    
    #Cleaning: keep only US data, extract state and lat/long
    print('Processing airports data...')
    df_airports_us_clean = df_airports.filter("iso_country == 'US'")\
        .withColumn("state", split(col("iso_region"), "-")[1])\
        .withColumn("latitude", split(col("coordinates"), ",")[0].cast(Dbl()))\
        .withColumn("longitude", split(col("coordinates"), ",")[1].cast(Dbl()))
    
    dim_airports = df_airports_us_clean.\
        drop('continent', 'iso_region', 'coordinates')
    
    # Write dim table into S3 in parquet format
    print('Writing airports dimension table in parquet...')
    
    dim_airports.write.partitionBy("state").parquet((output_data+"dim_airports"),'overwrite')
    print('Dim airports table created in parquet')


In [11]:
#spark = create_spark_session()
input_data = 'airport-codes_csv.csv'
output_data = "2016_04/"
process_airports_data(spark=spark, input_data=input_data, output_data=output_data)

Loading airport data...
Processing airports data...
Writing airports dimension table in parquet...
Dim airports table created in parquet


In [12]:
#read dim airports table 
table=spark.read.parquet("2016_04/dim_airports")
print(table.count())
table.show(3)

22757
+-----+-------------+--------------------+------------+-----------+------------+--------+---------+----------+------------------+------------------+-----+
|ident|         type|                name|elevation_ft|iso_country|municipality|gps_code|iata_code|local_code|          latitude|         longitude|state|
+-----+-------------+--------------------+------------+-----------+------------+--------+---------+----------+------------------+------------------+-----+
| 00TA|     heliport|Sw Region Faa Hel...|         598|         US|  Fort Worth|    00TA|     null|      00TA|-97.30580139160156|32.826900482177734|   TX|
| 00TE|     heliport|Tcjc-Northeast Ca...|         600|         US|  Fort Worth|    00TE|     null|      00TE|-97.18949890136719|32.847599029541016|   TX|
| 00TS|small_airport|Alpine Range Airport|         670|         US|     Everman|    00TS|     null|      00TS|-97.24199676513672|32.607601165771484|   TX|
+-----+-------------+--------------------+------------+---------

In [19]:
def process_cities_data(spark, input_data, output_data):
    """Extract US cities demographics data from provided folder in local machine, process it using Spark and 
    loads it into S3 as a dimension table in parquet format:
    @spark: spark session defined previously
    @input_data: location path for the dataset
    @output_data: S3 bucket name where to load processed output data
    """
        
    # Read in US cities data
    print('Loading cities data...')
    df_demographics = spark.read.csv(input_data, header = 'True', sep = ";")
    #pd.read_csv(input_data, sep=';')
    
    #Cleaning: no cleaning needed
    print('Processing cities data...')
    df_demographics_clean = df_demographics.select(
        col('City').alias('city'),
        col('State').alias('state_name'),
        col('Median Age').alias('median_age'),
        col('Male Population').alias('male_population'),
        col('Female Population').alias('female_population'),
        col('Number of Veterans').alias('number_veterans'),
        col('Foreign-born').alias('foreign_born'),
        col('Average Household Size').alias('avg_household_size'),
        col('State Code').alias('state_code'),
        col('Race').alias('race'),
        col('Count').alias('count'),
    )

    
    dim_cities_demographics = df_demographics_clean
    
    # Write dim table into S3 in parquet format
    print('Writing cities dimension table in parquet...')
    
    dim_cities_demographics.write.partitionBy("state_code").parquet((output_data+"dim_cities_demographics"),'overwrite')
    print('Dim cities table created in parquet')

In [20]:
#spark = create_spark_session()
input_data = 'us-cities-demographics.csv'
output_data = "2016_04/"
process_cities_data(spark=spark, input_data=input_data, output_data=output_data)

Loading cities data...
Processing cities data...
Writing cities dimension table in parquet...
Dim cities table created in parquet


In [21]:
#read dim cities table 
table=spark.read.parquet("2016_04/dim_cities_demographics")
print(table.count())
table.show(3)

2891
+----------------+----------+----------+---------------+-----------------+---------------+------------+------------------+--------------------+-----+----------+
|            city|state_name|median_age|male_population|female_population|number_veterans|foreign_born|avg_household_size|                race|count|state_code|
+----------------+----------+----------+---------------+-----------------+---------------+------------+------------------+--------------------+-----+----------+
|Rancho Cucamonga|California|      34.5|          88127|            87105|           5821|       33878|              3.18|Black or African-...|24437|        CA|
|     West Covina|California|      39.8|          51629|            56860|           3800|       37038|              3.56|               Asian|32716|        CA|
|          Folsom|California|      40.9|          41051|            35317|           4187|       13234|              2.62|  Hispanic or Latino| 5822|        CA|
+----------------+----------+

In [4]:
def process_temperature_data(spark, input_data, output_data):
    """Extract world temperature data from provided folder in local machine, process it using Spark and 
    loads it into S3 as a dimension table in parquet format:
    @spark: spark session defined previously
    @input_data: location path for the dataset
    @output_data: S3 bucket name where to load processed output data
    """
        
    # Read in US cities data
    print('Loading temperature data...')
    df_temperature = spark.read.csv(input_data, header = 'True')
    df_temperature.createOrReplaceTempView("df_temperature")
    
    #Cleaning: remove missing data, unnecessary fields and dates and compute avg. statistics by date/city/country
    print('Processing temperature data...')
    df_temperature_clean = spark.sql("""
        SELECT 
            dt as date,
            City as city,
            Country as country,
            avg(AverageTemperature) as avg_temperature,
            avg(AverageTemperatureUncertainty) as avg_temperature_uncertainty
            --count(*)
        FROM df_temperature
        WHERE 1=1
            and dt >='1960-01-01'
            --and dt >= (select min(date) from dim_time) --make sure we cover all dates we have in the fact table
            and AverageTemperature is not null
        group by date, city, country
    """)

    
    dim_world_temperatures = df_temperature_clean
    
    # Write dim table into S3 in parquet format
    print('Writing temperature dimension table in parquet...')
    
    dim_world_temperatures.write.partitionBy("country").parquet((output_data+"dim_world_temperatures"),'overwrite')
    print('Dim temperatures table created in parquet')

In [6]:
spark = create_spark_session()
input_data = '../../data2/GlobalLandTemperaturesByCity.csv'
output_data = "2016_04/"
process_temperature_data(spark=spark, input_data=input_data, output_data=output_data)

Loading temperature data...
Processing temperature data...
Writing temperature dimension table in parquet...
Dim temperatures table created in parquet


In [7]:
#read dim temperatures table 
table=spark.read.parquet("2016_04/dim_world_temperatures")
print(table.count())
table.show(3)

2247991
+----------+------+---------------+---------------------------+-------+
|      date|  city|avg_temperature|avg_temperature_uncertainty|country|
+----------+------+---------------+---------------------------+-------+
|1979-04-01|Abohar|         28.851|        0.28300000000000003|  India|
|1983-03-01|Abohar|         20.208|                      0.179|  India|
|1991-06-01|Abohar|         35.069|                      0.469|  India|
+----------+------+---------------+---------------------------+-------+
only showing top 3 rows



#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [32]:
def run_quality_checks():
    tables = ['fact_immigration', 'dim_time', 'dim_airports', 'dim_cities_demographics', 'dim_world_temperatures']
    for i in tables:
        print(i)
        path = "2016_04/"+ i
        #print(path)
        count_records = spark.read.parquet(path).count()
        #print(count_records)
        if count_records < 1:
            raise ValueError(f"Data quality check failed. {i} contained 0 rows")
        print(f"Data quality check on table {i} passed with {count_records} records") 
        print("Showing schema:")
        spark.read.parquet(path).printSchema()

In [33]:
run_quality_checks()

fact_immigration
Data quality check on table fact_immigration passed with 2953435 records
Showing schema:
root
 |-- citizenship_country: double (nullable = true)
 |-- residence_country: double (nullable = true)
 |-- port: string (nullable = true)
 |-- arrival_date: date (nullable = true)
 |-- arrival_mode: string (nullable = true)
 |-- departure_date: date (nullable = true)
 |-- respondent_age: double (nullable = true)
 |-- visa: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- visa_issued_department: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- arrival_flag: string (nullable = true)
 |-- departure_flag: string (nullable = true)
 |-- update_flag: string (nullable = true)
 |-- match_arrival_departure_fag: string (nullable = true)
 |-- birth_year: double (nullable = true)
 |-- allowed_to_stay_date: string (nullable = true)
 |-- ins_number: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admission_number: double (nulla

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

- The dictionary is provided inside **data_dictionary.txt** file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

- Given the already big size of both immigration data (about 3M rows) and world temperature dataset (8M rows), we decided to go for a leading big data framework such as **Apache Spark**. Also, thinking about the potential future increase in the amount of data (new immigrants records and temperature registrations), a technology such Spark will enable us to build ETL pipelines taking advantage of all the distributed processing techniques with an efficient use of memory.
- Moreover, being already familiar with SQL and Python's Pandas library, the SparkSQL module was a great incentive for choosing this technology. 
<br/>
- In terms of storage we chose to store our final fact/dimension table in **Amazon S3** in Parquet format, which offer an easy & flexible solution for hosting a Data Lake. 

Other possible scenarios to consider:  
- data increases by 100x: we would use a cluster like Amazon EMR and probably increase the number of nodes accordingly. We would still use Apache Spark for data processing and a service like Amazon S3 for storage.
- data populates a dashboard that must be updated on a daily basis by 7am every day: we would create the data pipeline with a tool like Apache Airflow which will help us coordinating and scheduling all the data processing steps as well as monitoring data quality. The data that will require a daily scheduling will probably be the immigration and possibly the world temperatures. While for cities and airports the updates will be much less frequent.
- database needed to be accessed by 100+ people: with current tables stored and partitioned as they are in S3 it should be an issue.